In [1]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg, select_category_value_agg, cnt_encoding
from mv_wg_avg import exp_weight_avg
from target_encoding import target_encoding

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals, impute_avg
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy, make_feature_set
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID_BUREAU'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV', 'SK_ID_BUREAU']

2018-07-24 23:40:26,489 logger 19 [INFO]    [logger_func] start 
2018-07-24 23:40:26,508 logger 19 [INFO]    [logger_func] start 
2018-07-24 23:40:26,508 logger 19 [INFO]    [logger_func] start 


In [3]:
# ' app '
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
# app = pd.read_csv('../data/application_train_test.csv')

# ' NULL埋め '
# app['DAYS_EMPLOYED'].fillna(-240, inplace=True)
# app['OCCUPATION_TYPE'].fillna('XNA', inplace=True)

# flag = {'N':0, 'Y':1}
# app['FLAG_OWN_CAR'] = app['FLAG_OWN_CAR'].map(flag)
# app['FLAG_OWN_REALTY'] = app['FLAG_OWN_REALTY'].map(flag)

' prev '
# prev.sort_values(by=[unique_id, 'SK_ID_prev', 'DAYS_CREDIT'], ascending=False, inplace=True)
# prev = row_number(prev, unique_id)

In [12]:
def category_summary(data):

    seller_code = {
        'Autotechnology': 'high_seller',
        'Connectivity': 'high_seller',
        'Jewelry': 'high_seller',
        'XNA': 'high_seller',

        'Consumerelectronics': 'low_seller',
        'Industry': 'low_seller',
        'Furniture': 'low_seller',
        'Clothing': 'low_seller',
        'Construction': 'low_seller',
        'MLMpartners': 'low_seller',
        'Tourism': 'low_seller'
    }
    data['SELLER_CODE'] = data['NAME_SELLER_INDUSTRY'].map(seller_code)

    purpose_code = {
        'Hobby': 'high_risk_purp',
        'Car repairs': 'high_risk_purp',
        'Refusal to name the goal': 'high_risk_purp',
        'Money for a third person': 'high_risk_purp',
        'Gasification / water supply': 'high_risk_purp',
        'Payments on other loans': 'high_risk_purp',
        'Urgent needs': 'high_risk_purp',
        'Building a house or an annex': 'high_risk_purp',
        'Medicine': 'high_risk_purp',
        'Wedding / gift / holiday': 'high_risk_purp',
        'Buying a used car': 'high_risk_purp',
        'Furniture': 'high_risk_purp',
        'Repairs': 'high_risk_purp',
        'Business development': 'high_risk_purp',
        'Other': 'high_risk_purp',
        'Purchase of electronic equipment': 'high_risk_purp',
        'Buying a holiday home / land': 'high_risk_purp',
        'Education': 'high_risk_purp',
        'Journey': 'high_risk_purp',
        'Everyday expenses': 'high_risk_purp',

        'XAP': 'low_risk_purp',
        'XNA': 'low_risk_purp',
        'Buying a garage': 'low_risk_purp',
        'Buying a new car': 'low_risk_purp',
        'Buying a home': 'low_risk_purp',


    }
    data['PURPOSE_CODE'] = data['NAME_CASH_LOAN_PURPOSE'].map(purpose_code)

    combi_code = {
        'Cash Street: middle': 'high_procombi',
        'Cash X-Sell: high': 'high_procombi',
        'Cash Street: high': 'high_procombi',
        'Card Street': 'high_procombi',

        'Cash Street: low': 'middle_procombi',
        'Cash': 'middle_procombi',
        'Card X-Sell': 'middle_procombi',
        'POS mobile with interest': 'middle_procombi',

        'POS other with interest': 'low_procombi',
        'POS mobile without interest': 'low_procombi',
        'Cash X-Sell: low': 'low_procombi',
        'POS household with interest': 'low_procombi',
        'POS others without interest': 'low_procombi',

        'POS household without interest': 'procombi',
        'Cash X-Sell: low': 'procombi',
        'POS industry with interest': 'procombi',
        'POS industry without interest': 'procombi',
    }
    data['COMBI_CODE'] = data['PRODUCT_COMBINATION'].map(combi_code)

    goods_code = {
        'Insurance': 'high_goods',
        'Vehicles': 'high_goods',
        'XNA': 'high_goods',
        'Auto Accessories': 'high_goods',
        'Jewelry': 'high_goods',
        'Mobile': 'high_goods',
        'Office Appliances': 'high_goods',
        'Direct Sales': 'high_goods',

        'Computers': 'middle_goods',
        'Audio/Video': 'middle_goods',
        'Photo /CinemaEquipment': 'middle_goods',
        'Sport and Leisure': 'middle_goods',
        'Consumer Electronics': 'middle_goods',
        'Construction Materials': 'middle_goods',
        'Gardening': 'middle_goods',
        'Homewares': 'middle_goods',
        
        'Additional Service': 'middle_goods',
        'Medicine': 'middle_goods',
        'Weapon': 'middle_goods',
        'Furniture': 'middle_goods',
        'other': 'middle_goods',
        'clothing and Accessories': 'middle_goods',
        'education': 'middle_goods',
        'medical Supplies': 'middle_goods',
        'tourism': 'middle_goods',
        'fitness': 'middle_goods',
        'animals': 'middle_goods'

#         'Additional Service': 'low_goods',
#         'Medicine': 'low_goods',
#         'Weapon': 'low_goods',
#         'Furniture': 'low_goods',
#         'other': 'low_goods',
#         'clothing and Accessories': 'low_goods',
#         'education': 'low_goods',
#         'medical Supplies': 'low_goods',
#         'tourism': 'low_goods',
#         'fitness': 'low_goods',
#         'animals': 'low_goods'
    }
    data['GOODS_CODE'] = data['NAME_GOODS_CATEGORY'].map(goods_code)

    reject_code = {
#         'SCOFR': 'high_reject',
        'SCOFR': 'middle_reject',

        'LIMIT': 'middle_reject',
        'HC': 'middle_reject',
        'XNA': 'middle_reject',
        'SCO': 'middle_reject',
        'VERIF': 'middle_reject',

        'CLIENT': 'low_reject',
        'XAP': 'low_reject',
        'SYSTEM': 'low_reject',
    }
    data['REJECT_CODE'] = data['CODE_REJECT_REASON'].map(reject_code)

    ' 変更なし '
    data['PORTFOLIO_CODE'] = data['NAME_PORTFOLIO'].map(
        lambda x: 'POS' if x == 'Cars' else 'Cards' if x == 'XNA' else x)

    ' 変更なし '
    data['YIELD_CODE'] = data['NAME_YIELD_GROUP'].map(
        lambda x: 'low_normal' if x == 'low_action' else x)

    ' 変更なし '
    data['CONTRACT_CODE'] = data['NAME_CONTRACT_TYPE'].map(
        lambda x: 'Revolving loans' if x == 'XNA' else x)

    ' 変更なし '
    data['CHANNEL_CODE'] = data['CHANNEL_TYPE'].map(
        lambda x: 'Stone' if x.count('corporate') or x.count('dealer') or x.count('Regional') else 'Contact center' if x.count('AP') else x)

    return data

prev = pd.read_csv('../data/previous_cleansing.csv')
prev = category_summary(prev)
df = prev

' 可視化 '
cat_list = ['SELLER_CODE', 'PURPOSE_CODE', 'GOODS_CODE', 'REJECT_CODE', 'PORTFOLIO_CODE', 'YIELD_CODE', 'CONTRACT_CODE', 'CHANNEL_CODE', 'COMBI_CODE']
for category in cat_list:
    val_cnt = df[category].value_counts().to_frame()
    df = df.query('TARGET>=0')
    positive = df.groupby(category)[target].mean().to_frame()
    display(val_cnt.join(positive))

,SELLER_CODE,TARGET
high_seller,1134458,0.093174
low_seller,131286,0.063082


,PURPOSE_CODE,TARGET
low_risk_purp,1355991,0.084655
high_risk_purp,57710,0.131138


,GOODS_CODE,TARGET
high_goods,1007595,0.092393
middle_goods,357652,0.072998


,REJECT_CODE,TARGET
low_reject,1168976,0.079526
middle_reject,244725,0.120119


,PORTFOLIO_CODE,TARGET
POS,592840,0.076346
Cards,429198,0.098633
Cash,391663,0.088765


,YIELD_CODE,TARGET
XNA,429198,0.098633
low_normal,353471,0.069808
middle,324971,0.080432
high,306061,0.095452


,CONTRACT_CODE,TARGET
Cash loans,626764,0.091258
Consumer loans,625256,0.077100
Revolving loans,161681,0.104873


,CHANNEL_CODE,TARGET
Credit and cash offices,600342,0.090110
Country-wide,423062,0.082560
Stone,281782,0.074249
Contact center,108515,0.114390


,COMBI_CODE,TARGET
middle_procombi,521318,0.092740
procombi,276247,0.064384
low_procombi,268758,0.077717
high_procombi,227029,0.113955


In [8]:
path = '../features/5_engineering/*.npy'
df = make_feature_set(base, path)

'positive率とサンプル数によってラベリングする'

' SUITE'
df['a_NAME_TYPE_SUITE'] = df['a_NAME_TYPE_SUITE'].map({
    'Children' : 'middle_row_risk'
    ,'Family' : 'middle_row_risk'
    ,'Unaccompanied' : 'middle_risk'
    ,'Groupofpeople' : 'hight_risk'
    ,'Other_A' : 'high_risk'
    ,'Other_B' : 'high_risk'
    ,'Spouse,partner' : 'row_risk'
})

' INCOME_TYPE'
df['a_NAME_INCOME_TYPE'] = df['a_NAME_INCOME_TYPE'].map(lambda x:'Pensioner' if x.count('Business') or x.count('Student') else 'Working' if x.count('Unemployed') or x.count('Maternity') else x)

' EDUCATION_TYPE'
df['a_NAME_EDUCATION_TYPE'] = df['a_NAME_EDUCATION_TYPE'].map(lambda x:'Secondary / secondary special' if x.count('Lower secondary') else 'Higher education' if x.count('Academic') else x)

' EDUCATION_TYPE'
df['a_NAME_FAMILY_STATUS'] = df['a_NAME_FAMILY_STATUS'].map(lambda x:'Married' if x.count('Widow') or x.count('Unknown') else 'Civil marriage' if x.count('Single') or x.count('Separated') else x)

' ORGANIZATION '
tmp = df.query('TARGET != -1')
category = 'a_ORGANIZATION_TYPE'
# category = 'a_OCCUPATION_TYPE'
target_avg = tmp.groupby(category)['TARGET'].mean().reset_index()
# print(target_avg)
# sys.exit()

' positive率でカテゴリをビンにする '
bins = 10
bin_list = ['TARGET']
for col in bin_list:
    target_avg[target] = pd.qcut(x=target_avg[col], q=bins, duplicates='drop')
    target_avg.set_index(category, inplace=True)
    target_avg = target_avg.iloc[:,0]
df[category] = df[category].map(target_avg)

' positive率が低い順に番号をつける '
bin_sort = df[category].drop_duplicates().sort_values().to_frame()
bin_sort['index'] = np.arange(len(bin_sort))+1
bin_sort = bin_sort.set_index(category).iloc[:,0]
df[category] = df[category].map(bin_sort)

df.head()
fe = df.copy()
del df

,SK_ID_CURR,AMT_CREDIT_diff_AMT_ANNUITY@_div_AMT_ANNUITY@,AMT_CREDIT_diff_AMT_ANNUITY@_div_AMT_GOODS_PRICE@,a_AMT_CREDIT,a_AMT_INCOME_TOTAL,a_CODE_GENDER,a_DAYS_BIRTH,a_DAYS_EMPLOYED,a_DAYS_ID_PUBLISH,a_DAYS_LAST_PHONE_CHANGE,a_DAYS_REGISTRATION,a_EXT_SOURCE_avg@,a_FLAG_WORK_PHONE,a_HOUSE_HOLD_CODE@,a_NAME_EDUCATION_TYPE,a_NAME_FAMILY_STATUS,a_NAME_INCOME_TYPE,a_NAME_TYPE_SUITE,a_OCCUPATION_TYPE,a_ORGANIZATION_TYPE,a_REGION_RATING_CLIENT_W_CITY,is_test,is_train,TARGET,valid_no_4
0,100002,15.461104,1.088026,406597.5,202500.0,M,-315,-21.0,-70,-37.0,-121,0.161787,0,1,Secondary / secondary special,Civil marriage,Working,middle_risk,8,8,2,0,1,1,2.0
1,100003,35.234085,1.113594,1293502.5,270000.0,F,-558,-39.0,-9,-27.0,-39,0.478034,0,-2,Higher education,Married,State servant,middle_row_risk,2,3,1,0,1,0,1.0
2,100004,19.000000,0.950000,135000.0,67500.0,M,-634,-7.0,-84,-27.0,-142,0.562556,1,1,Secondary / secondary special,Civil marriage,Working,middle_risk,8,4,2,0,1,0,5.0
3,100006,9.532818,0.952848,312682.5,135000.0,F,-633,-101.0,-81,-20.0,-327,0.560823,0,-2,Secondary / secondary special,Civil marriage,Working,middle_risk,8,8,2,0,1,0,3.0
4,100007,22.461618,0.957377,513000.0,121500.0,M,-664,-101.0,-115,-36.0,-143,0.421563,0,1,Secondary / secondary special,Civil marriage,Working,middle_risk,2,3,2,0,1,0,4.0


In [11]:
num_list = get_numeric_features(data=prev, ignore=ignore_features)
num_list = 
print(num_list)

['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY', 'STATUS', 'DAYS_ENDDATE', 'CREDIT_TERM', 'LOAN_MONTHLY', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@', 'LOAN_MONTHLY_diff_AMT_ANNUITY@']


In [12]:
bureau.sort_values(by=[unique_id, 'SK_ID_BUREAU', 'DAYS_CREDIT'], ascending=False, inplace=True)
bureau = row_number(bureau, [unique_id, 'CREDIT_TYPE'])
bureau.head(40)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET,over_flg,progress_flg,row_no,STATUS,DAYS_ENDDATE,CREDIT_TERM,LOAN_MONTHLY,AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@,LOAN_MONTHLY_diff_AMT_ANNUITY@
0,456253,6230038,Active,currency 1,-23.0,NaN,37.0,NaN,NaN,NaN,2250000.00,1529545.5,0.0,NaN,middle_CREDIT_TYPE,0.0,NaN,0.0,0,1,1,1.0,37.0,60.0,3.750000e+04,720454.50,NaN
1,456253,6098499,Closed,currency 1,-30.0,NaN,-6.0,-26.0,NaN,NaN,675000.00,0.0,0.0,NaN,middle_CREDIT_TYPE,-23.0,58369.500,0.0,0,0,2,1.0,-26.0,4.0,1.687500e+05,675000.00,1.103805e+05
2,456253,6098498,Closed,currency 1,-30.0,NaN,-6.0,-26.0,NaN,NaN,675000.00,0.0,0.0,NaN,middle_CREDIT_TYPE,-4.0,58369.500,0.0,0,0,3,1.0,-26.0,4.0,1.687500e+05,675000.00,1.103805e+05
3,456253,5113039,Active,currency 1,-30.0,NaN,12.0,NaN,NaN,NaN,360000.00,171036.0,0.0,NaN,high_CREDIT_TYPE,-5.0,58369.500,0.0,0,1,1,1.0,12.0,42.0,8.571429e+03,188964.00,-4.979807e+04
4,456249,5371702,Closed,currency 1,-41.0,NaN,-30.0,-30.0,NaN,NaN,540000.00,NaN,NaN,NaN,middle_CREDIT_TYPE,-30.0,NaN,0.0,0,0,1,NaN,-30.0,11.0,4.909091e+04,NaN,NaN
5,456249,5371701,Closed,currency 1,-49.0,NaN,-37.0,-38.0,NaN,NaN,450000.00,NaN,NaN,NaN,middle_CREDIT_TYPE,-38.0,NaN,0.0,0,0,2,NaN,-38.0,11.0,4.090909e+04,NaN,NaN
6,456249,5371700,Closed,currency 1,-63.0,NaN,-57.0,-58.0,NaN,NaN,248692.50,0.0,NaN,NaN,middle_CREDIT_TYPE,-57.0,NaN,0.0,0,0,3,NaN,-58.0,5.0,4.973850e+04,248692.50,NaN
7,456249,5371696,Closed,currency 1,-48.0,NaN,-39.0,-41.0,18945.000,NaN,120183.66,0.0,0.0,NaN,middle_CREDIT_TYPE,-41.0,NaN,0.0,1,0,4,NaN,-41.0,7.0,1.716909e+04,120183.66,NaN
8,456247,6022156,Closed,currency 1,-52.0,NaN,-28.0,-39.0,NaN,NaN,292500.00,NaN,NaN,NaN,middle_CREDIT_TYPE,-39.0,NaN,0.0,0,0,1,1.0,-39.0,13.0,2.250000e+04,NaN,NaN
9,456247,6022155,Closed,currency 1,-27.0,NaN,-26.0,-26.0,NaN,NaN,101250.00,0.0,0.0,NaN,middle_CREDIT_TYPE,-26.0,NaN,0.0,0,0,2,1.0,-26.0,1.0,1.012500e+05,101250.00,NaN


In [ ]:

def bureau_extract_agg(data, bins=0, val='', status='', term='', freq=0):

    if freq>0:
        prefix = f'b_{val}_{status}_latest{freq}_'
    elif freq==0:
        prefix = f'b_{val}_{status}_{term}_'
    
    ' カテゴリー区分 '
    if val.count('high'):
        data = data.query(f"CREDIT_TYPE=='{val}'")
    elif val.count('middle'):
        data = data.query(f"CREDIT_TYPE=='{val}'")
    elif val.count('row'):
        data = data.query(f"CREDIT_TYPE=='{val}'")
    
    ' ステータス区分 '
    if status.count('Act'):
        data = data.query(f"CREDIT_ACTIVE=='{status}'")
    elif status.count('Clo'):
        data = data.query(f"CREDIT_ACTIVE=='{status}'")
    elif status.count('og_a'):
        data = data.query(f"DAYS_ENDDATE!=DAYS_ENDDATE or DAYS_ENDDATE>0")
    elif status.count('og_c'):
        data = data.query(f"DAYS_ENDDATE<=0")
        
    ' 期間 '
    if term.count('3mon'):
        data = data.query(f"DAYS_CREDIT>=-3")
    elif term.count('6mon'):
        data = data.query(f"DAYS_CREDIT>=-6")
    elif term.count('1year'):
        data = data.query(f"DAYS_CREDIT>=-12")
    elif term.count('2year'):
        data = data.query(f"DAYS_CREDIT>=-24")
    elif term.count('3year'):
        data = data.query(f"DAYS_CREDIT>=-36")
        
    if freq>0:
        data = data.query(f'row_no<={freq}')
    
    ' dataチェック '
    print(data.shape)
    print('*** SK_ID_CURR ***')
    print(data[unique_id].drop_duplicates().shape)
    if data[unique_id].drop_duplicates().count()<10000:
        return
    # sys.exit()
    
    ext = data.copy()
    del data
    gc.collect()
    
    
    days_list = [col for col in ext.columns if col.count('DAYS')]
    amt_list = [col for col in ext.columns if col.count('AMT_')]
    # print(days_list)
    # print(amt_list)
    
    amt = ext[[unique_id] + amt_list].groupby(unique_id).agg('sum')
    days = ext[[unique_id] + days_list].groupby(unique_id).agg('max')
    result = amt.join(days)
    
    for col in result.columns:
        if col in ignore_features:
            continue
        elif col.count('AMT_'):
            result.rename(columns={col:f'{col}_sum@'}, inplace=True)
        elif col.count('DAYS_'):
            result.rename(columns={col:f'{col}_max@'}, inplace=True)
            
    ' ここからターゲットエンコーディング '
    method_list = ['mean', 'std']
    # method_list = ['sum', 'mean', 'std', 'max', 'min']
    
    val_col = 'valid_no_4'
    bin_list  = [col for col in result.columns if col.count('@')]
    result = result.merge(fe, on=[unique_id], how='inner')
    
    cat_list_1 = []
    # cat_list_1 = [
    #     'a_OCCUPATION_TYPE'
    #     ,'a_ORGANIZATION_TYPE'
    #     ,'a_NAME_EDUCATION_TYPE'
    #     ,'a_NAME_INCOME_TYPE'
    #     ,'a_CODE_GENDER'
    #     ,'a_NAME_TYPE_SUITE'
    #     ,'a_NAME_FAMILY_STATUS'
    #     ,'a_REGION_RATING_CLIENT_W_CITY'
    #     ,'a_HOUSE_HOLD_CODE@'
    #     ,'a_FLAG_WORK_PHONE'
    # ]
    
    cat_list_2 = [
        'a_OCCUPATION_TYPE'
        ,'a_ORGANIZATION_TYPE'
        ,'a_NAME_EDUCATION_TYPE'
        ,'a_NAME_INCOME_TYPE'
        ,'a_CODE_GENDER'
        ,'a_NAME_FAMILY_STATUS'
        ,'a_NAME_TYPE_SUITE'
        ,'a_REGION_RATING_CLIENT_W_CITY'
        ,'a_HOUSE_HOLD_CODE@'
        ,'a_FLAG_WORK_PHONE'
    ]
    
    ' binをカテゴリとする場合はこちら '
    for col in bin_list:
        print(col)
        if len(result[col].drop_duplicates())<bins: return
        result[f'bin{bins}_{col}'] = pd.qcut(x=result[col], q=bins, duplicates='drop')
        cat_list_1.append(f'bin{bins}_{col}')
        
    #     ' 可視化チェック '
    #     print(result[f'bin{bins}_{col}'].value_counts())
    #     result.drop(f'bin{bins}_{col}', axis=1, inplace=True)
    #     sys.exit()
        
    
    categorical = []
    for cat1 in cat_list_1:
        for cat2 in cat_list_2:
            if cat1==cat2:continue
            categorical.append([cat1, cat2])
    
    ' エンコーディングするカテゴリカラムのリストを作る '
    #  categorical = get_categorical_features(result, ignore_features)
    
    ' カテゴリの組み合わせをエンコーディングする場合はこちら '
    #  cat_combi = list(combinations(categorical, 2))
    #  categorical = cat_combi
    
    select_list = []
    #  categorical = [col for col in result.columns if col.count('cluster')]
    
    for cat in categorical:
        target_encoding(base=base, data=result, unique_id=unique_id, level=cat, method_list=method_list, prefix=prefix, select_list=select_list, test=1, impute=1208, val_col=val_col)
    

'集計前の絞り込み条件'
'SK_ID_CURRが1万件以上のデータに限る'
data = bureau.copy()
bins = 30
bins_list = [20, 10, 5]

val = ''
# val = 'high_CREDIT_TYPE'
# val = 'middle_CREDIT_TYPE'
# val = 'row_CREDIT_TYPE'
val_list = ['', 'high_CREDIT_TYPE' ,'middle_CREDIT_TYPE' ,'row_CREDIT_TYPE']
# val_list = ['']

status = ''
# status = 'Active'
# status = 'Closed'
# status = 'og_a'
# status = 'og_c'
status_list = ['', 'Active' ,'Closed' ,'og_a' ,'og_c']

term = ''
# term = '3month'
# term = '6month'
# term = '1year'
# term = '2year'
# term = '3year'
term_list = ['3month' ,'6month' ,'1year' ,'2year' ,'3year']

freq = 0
# freq = 3
# freq = 5
# freq = 10
freq_list = [3, 5, 10]

number = 0
for bins in bins_list:
    for val in val_list:
        for status in status_list:
#             for term in term_list:
            for freq in freq_list:
                number += 1
                if number<10:
                    continue
                bureau_extract_agg(data=data, bins=bins, val=val, status=status, freq=freq)
            
for bins in bins_list:
    for val in val_list:
        for status in status_list:
            for term in term_list:
                bureau_extract_agg(data=data, bins=bins, val=val, status=status, term=term)

(473386, 27)
*** SK_ID_CURR ***
(176053,)


../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
